In [ ]:
# Notebook 6: Vendor Scorecard & Analytics for EthioMart
# ======================================================

# Install required packages
!pip install pandas numpy

import pandas as pd
import numpy as np

# ==============================
# 1. Load Scraped Telegram Data
# ==============================
# Example structure: each row is a Telegram message with metadata
# Columns: ['channel', 'message', 'views', 'timestamp', 'product', 'price', 'location']
data = pd.read_csv("data/processed/telegram_messages_with_entities.csv")

# ==============================
# 2. Compute Vendor Metrics
# ==============================
vendor_summary = []

for vendor, group in data.groupby("channel"):
    # Activity & Consistency
    posts_per_week = group.shape[0] / ((group['timestamp'].max() - group['timestamp'].min()).days / 7)

    # Market Reach & Engagement
    avg_views = group['views'].mean()
    top_post_idx = group['views'].idxmax()
    top_post = group.loc[top_post_idx]

    # Business Profile
    avg_price = group['price'].mean()

    # Simple Lending Score
    lending_score = (avg_views * 0.5) + (posts_per_week * 0.5)

    vendor_summary.append({
        "Vendor": vendor,
        "Posts/Week": round(posts_per_week, 2),
        "Avg. Views/Post": round(avg_views, 2),
        "Top Post Product": top_post['product'],
        "Top Post Price": top_post['price'],
        "Avg. Price (ETB)": round(avg_price, 2),
        "Lending Score": round(lending_score, 2)
    })

vendor_df = pd.DataFrame(vendor_summary)
vendor_df = vendor_df.sort_values(by="Lending Score", ascending=False)

# ==============================
# 3. Display Vendor Scorecard
# ==============================
print("Vendor Scorecard:")
print(vendor_df)

# ==============================
# 4. Save Scorecard
# ==============================
vendor_df.to_csv("data/vendor_scorecard.csv", index=False)
